In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.model_selection import train_test_split

from Strategy import *

In [34]:
df = pd.read_csv("Y:\\Dropbox\\Dropbox (MIT)\\Robinhood Trading\\Stock Data\\broader_stock.csv")

In [35]:
df = df.iloc[:-1]

In [36]:
for tick in df.columns[1:]:
    initpos = df[tick].first_valid_index()
    df[tick].iloc[initpos:] = df[tick].iloc[initpos:].interpolate()

In [38]:
df = df.set_index(pd.to_datetime(df['Date']))
df.drop(['Date'], axis=1, inplace=True)
pct_df = df.pct_change().shift(1).iloc[2:]

In [96]:
train_dta, test_dta = train_test_split(pct_df, test_size=0.01, shuffle=False)

### Partial out $R_m$ from individual return series
Using only the past 200 observations

In [251]:
residual = []

In [252]:
for tick in train_dta.columns[::3][:40]:
    rm = train_dta['SPY_Open']
    rint = df['^TNX_Open'].loc[train_dta.index]
    ri = train_dta[tick]
    temp = pd.concat([ri, rm, rint], axis=1).dropna().iloc[-500:]

    y = temp[temp.columns[0]].values
    X = temp[temp.columns[1:]].values

    reg = ElasticNetCV(cv=10, n_jobs=-1).fit(X, y)
    yhat = reg.predict(X)
    eps = y - yhat
    residual.append(eps)

In [259]:
RES = np.column_stack(residual)

In [243]:
reglm = LinearRegression().fit(X,y)

In [244]:
rm = test_dta['SPY_Open']
rint = df['^TNX_Open'].loc[test_dta.index]
ri = test_dta['V_Open']
temp_test = pd.concat([ri, rm, rint], axis=1).dropna()

In [245]:
y_test = temp_test[temp_test.columns[0]].values
X_test = temp_test[temp_test.columns[1:]].values

In [246]:
reg.score(X,y)

0.752122173966991

In [247]:
reglm.score(X, y)

0.7521238460816104

In [248]:
reg.score(X_test, y_test)

0.2583277874678218

In [249]:
reglm.score(X_test, y_test)

0.2582859063212264